In [46]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from random import sample

from src.utils import get_rules_list
from src.utils import load_data_label_vector

In [47]:
# Get data
import pandas as pd


# train = load_data_label_vector("train.csv")
# test = load_data_label_vector("test.csv")

train = load_data_label_vector("/app/wafcraft/data/prepared/2024-06-09_20-11-04_lightslategray-them/train.csv")
test = load_data_label_vector("/app/wafcraft/data/prepared/2024-06-09_20-11-04_lightslategray-them/test.csv")

# test = pd.concat(
#     [
#         load_data_label_vector(
#             "/app/wafcraft/data/prepared/2024-04-07_18-15-53_brown-lot/test_adv.csv"
#         ),
#         load_data_label_vector(
#             "/app/wafcraft/data/prepared/2024-04-07_18-15-53_brown-lot/test.csv"
#         ),
#     ],
#     ignore_index=True,
# )

In [48]:
# print data info

print(train.shape)
print(test.shape)

# print how many are in train with label 1
print("train with label 1")
print(train[train["label"] == 1].shape)
print("train with label 0")
print(train[train["label"] == 0].shape)

print("test with label 1")
print(test[test["label"] == 1].shape)
print("test with label 0")
print(test[test["label"] == 0].shape)

# check how much train and test overlap on the same data columns
print("overlap")


overlap = pd.merge(test, train, how="inner", on=["data"])
print(overlap.shape)
# print(overlap.head())

(20000, 3)
(4000, 3)
train with label 1
(10000, 3)
train with label 0
(10000, 3)
test with label 1
(2000, 3)
test with label 0
(2000, 3)
overlap
(1, 5)


In [49]:
# remove some features (simulating the paper's feature space)


# remove_feature_ids = sample(range(0, 61), 0)
remove_feature_ids = [43, 37, 22, 53, 50, 46, 47, 33, 8, 34, 21, 15]
# remove_feature_ids = []
print(remove_feature_ids)
print(len(remove_feature_ids))
print(f"features still: {len(train.iloc[1]['vector']) - len(remove_feature_ids)}")

def setzero(vector, i):
    vector[i] = 0
    return vector

for i in remove_feature_ids:
    train["vector"] = train["vector"].apply(lambda x: setzero(x, i))

for i in remove_feature_ids:
    test["vector"] = test["vector"].apply(lambda x: setzero(x, i))

[43, 37, 22, 53, 50, 46, 47, 33, 8, 34, 21, 15]
12
features still: 57


In [50]:
print(train["vector"][0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0.]


In [51]:
# train model
X_train, y_train = list(train["vector"]), train["label"]
X_test, y_test = list(test["vector"]), test["label"]

model = RandomForestClassifier()

model.fit(X_train, y_train)

RandomForestClassifier()

In [52]:
# Evaluate check model TPR at desired FPR
# calculate TPR at desired FPR
from sklearn.metrics import confusion_matrix, roc_curve


probabilities = model.predict_proba(X_test)[:, 1]

fpr, tpr, thresholds = roc_curve(y_test, probabilities)  # plot ROC curve
closest_idx = np.argmin(np.abs(fpr - 0.01))  # threshold closest to FPR
threshold = thresholds[closest_idx]
adjusted_predictions = (probabilities >= threshold).astype(int)  #  new preds

cm = confusion_matrix(y_test, adjusted_predictions)

tn, fp, fn, tp = cm.ravel()

tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(f"TPR: {tpr}")
print(f"FPR: {fpr}")

TPR: 0.895
FPR: 0.0095
